In [9]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import re



In [2]:
# Set whether to display selium browser
VISIBLE = True

options = Options()
# If not visible do not display browser to user
if not VISIBLE: options.add_argument('--headless')

url = "https://goblin.bet/#/"
driver = webdriver.Firefox(options=options)
driver.get(url)




In [3]:
contButt = WebDriverWait(driver, timeout=30).until(lambda d: d.find_element(By.CLASS_NAME, "WelcomeStart.Left"))
contButt.click()

In [14]:
pageSrc = driver.page_source
soup = BeautifulSoup(pageSrc)

RIGHT = True
LEFT = False

def getLog(soup: BeautifulSoup, printLog = True):
    # Find log
    log = soup.find("div", {"class": "scrollhost BLogScroll"})
    # If print log true print out the log
    if printLog:
        for lg in log.find_all("div", {"class": "LogText"}):
            print(lg.text)
    # return the betting log
    return log

def getMoney(soup: BeautifulSoup):
    return soup.find('span', {"class": "BetsScore"}).text

def getCreatureSoup(soup: BeautifulSoup, side=LEFT):
    if side == LEFT:
        return soup.find("div", {"class": "Block Statsheet Left TeamRed"})
    return soup.find("div", {"class": "Block Statsheet Right TeamBlue"})
        

In [43]:
soup = BeautifulSoup(driver.page_source)
log = getLog(soup)

+ Round begins in 30 seconds!
+ Challenger 2: Aboleth
+ Challenger 1: Young Black Dragon
+ ------NEW BATTLE------ +
+ Yggdrasill wins! Stone Golem is eliminated!
+ Yggdrasill resists Slowed (WIS) : SUCCESS
+ Stone Golem dies!
+ Stone Golem takes 11 Bludgeoning damage!
+ The Awakened Tree's Tree Slam crits!
+ Awakened Tree attacks Stone Golem with a Tree Slam.
+ Stone Golem takes 8 Bludgeoning damage!
+ The Awakened Tree's Tree Slam hits!
+ Awakened Tree attacks Stone Golem with a Tree Slam.
+ Stone Golem takes 22 Bludgeoning damage!
+ The Yggdrasill's Slam crits!
+ Yggdrasill attacks Stone Golem with a Slam.
+ Yggdrasill takes 12 Bludgeoning damage!
+ The Stone Golem's Slam hits!
+ Stone Golem attacks Yggdrasill with a Slam.
+ The Stone Golem's Slam misses!
+ Stone Golem attacks Yggdrasill with a Slam.
+ Yggdrasill resists Slowed (WIS) : FAILURE
+ Stone Golem takes 7 Bludgeoning damage!
+ The Awakened Tree's Tree Slam hits!
+ Awakened Tree attacks Stone Golem with a Tree Slam.
+ Stone 

In [44]:
statsList = soup.find("div", {"class": "SSStats"})
print(statsList.prettify())

<div class="SSStats">
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  ATTRIBUTES
 </span>
 <span class="Stat">
  STR: 19
 </span>
 <span class="Stat">
  DEX: 14
 </span>
 <span class="Stat">
  CON: 17
 </span>
 <span class="Stat">
  INT: 12
 </span>
 <span class="Stat">
  WIS: 11
 </span>
 <span class="Stat">
  CHA: 15
 </span>
 <span class="Stat Highlight">
  HP: 67/67
 </span>
 <span class="Stat Highlight">
  AC: 18
 </span>
 <span class="Stat Highlight">
  Spd: 80ft
 </span>
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  WINS
 </span>
 <span class="Stat Wide">
  Deva
 </span>
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  IMMUNE
 </span>
 <span class="Stat Mod">
  <span>
   Acid
   <span class="PiercedResist">
   </span>
  </span>
 </span>
 <span class="SSHeader">
  <img class="SSHeaderDots" src="/Art/UI/SSHeaderDots.png"/>
  ACTIONS
 </span>
 <div class="Stat Wi

In [42]:
def stripChrs(word):
    return re.sub("[^0-9]", '', word)


class Creature:
    def __init__(self, soup: BeautifulSoup, side=LEFT):
        self.soup = getCreatureSoup(soup, side)
        self.name = self.InitName()
        
    def InitName(self):
        return self.soup.find("span", {"class": "SSName"}).text
    
    def InitACAR(self):
        self.cr = re.sub("[^0-9]", '', self.soup.find("span", {"class": "SSInfo"}).text)

    def InitImmune(self):
        immunities = []
        for i, stat in enumerate(statsList = self.soup.find("div", {"class": "SSStats"})):


    def getAttributes(self):
        stats = {"STR": None, "DEX": None, "CON": None, "INT": None, "WIS": None, "CHA": None}
        statsList = self.soup.find("div", {"class": "SSStats"})

        for i, stat in enumerate(statsList.find_all("span", {"class": "Stat"})):
            if i == 0:
                stats["STR"] = stripChrs(stat.text)
            elif i == 1:
                stats["DEX"] = stripChrs(stat.text)
            elif i == 2:
                stats["CON"] = stripChrs(stat.text)
            elif i == 3:
                stats["INT"] = stripChrs(stat.text)
            elif i == 4:
                stats["WIS"] = stripChrs(stat.text)
            elif i == 5:
                stats["CHA"] = stripChrs(stat.text)
            elif i == 6:
                stats["HP_Ratio"] = stripChrs(stat.text)
            elif i == 7:
                stats["AC"] = stripChrs(stat.text)
            elif i == 8:
                stats["SPD"] = stripChrs(stat.text)
        # Assign Stats
        #stats["STR"] = statsList[0] #re.sub("[^0-9]", '', statsList[0].text)

cret = Creature(soup, RIGHT)


IndentationError: expected an indented block after 'elif' statement on line 38 (3205846722.py, line 39)

In [41]:
cret.name
cret.getAttributes()

Spirit Naga
Gauntlets of Ogre PowerHas 19 STR
On Fire
Dead
19


In [18]:
r = requests.get("https://goblin.bet/#/").text
soup = BeautifulSoup(r)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="Monsters fight for your pleasure, throw gold coins at them" name="description"/>
  <title>
   Goblin Bet
  </title>
  <link href="/static/css/main.bc1c7931.chunk.css" rel="stylesheet"/>
 </head>
 <body>
  <noscript>
   You need to enable JavaScript to run this app.
  </noscript>
  <div id="root">
  </div>
  <script>
   !function(e){function t(t){for(var n,l,i=t[0],f=t[1],a=t[2],c=0,s=[];c<i.length;c++)l=i[c],Object.prototype.hasOwnProperty.call(o,l)&&o[l]&&s.push(o[l][0]),o[l]=0;for(n in f)Object.prototype.hasOwnProperty.call(f,n)&&(e[n]=f[n]);for(p&&p(t);s.length;)s.shift()();return u.push.apply(u,a||[]),r()}function r(){for(var e,t=0;t<u.length;t++){for(var r=u[t],n=!0,i=1;i<r.length;i++){var f=r[i];0!==o[f]&&(n=!1)}n&&(u.splice(t--,1),e=l(l.s=r[0]))}return e}var n={},o={1:0},u=[];func